## Upotukset

Edellisessä esimerkissämme käsittelimme korkeulotteisia bag-of-words-vektoreita, joiden pituus oli `vocab_size`, ja muunsimme ne eksplisiittisesti matalaulotteisista sijaintiesityksistä harvaan yksi-kuuma-esitykseen. Tämä yksi-kuuma-esitys ei ole muistin kannalta tehokas, ja lisäksi jokainen sana käsitellään toisistaan riippumattomasti, eli yksi-kuuma-koodatut vektorit eivät ilmaise sanojen välistä semanttista samankaltaisuutta.

Tässä osiossa jatkamme **News AG** -aineiston tutkimista. Aloitetaan lataamalla data ja hakemalla joitakin määritelmiä edellisestä muistikirjasta.


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)
print("Vocab size = ",vocab_size)

Loading dataset...


d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\train.csv: 29.5MB [00:01, 18.8MB/s]                            
d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\test.csv: 1.86MB [00:00, 11.2MB/s]                          


Building vocab...
Vocab size =  95812


## Mikä on upotus?

**Upotuksen** idea on esittää sanoja matalampiulotteisina tiheinä vektoreina, jotka jollain tavalla heijastavat sanan semanttista merkitystä. Myöhemmin käsittelemme, kuinka rakentaa merkityksellisiä sanavektoreita, mutta toistaiseksi voimme ajatella upotuksia tapana pienentää sanavektorin ulottuvuuksia.

Upotuskerros ottaa sanan syötteenä ja tuottaa ulostulovektorin, jonka koko on määritelty `embedding_size`. Tietyssä mielessä se on hyvin samanlainen kuin `Linear`-kerros, mutta sen sijaan, että se ottaisi yhden kuuman koodatun vektorin, se pystyy ottamaan sanan numeron syötteenä.

Kun käytämme upotuskerrosta verkkomme ensimmäisenä kerroksena, voimme siirtyä bag-of-words-mallista **embedding bag** -malliin, jossa ensin muutamme jokaisen tekstimme sanan vastaavaksi upotukseksi ja sitten laskemme jonkin aggregaattifunktion kaikkien näiden upotusten yli, kuten `sum`, `average` tai `max`.

![Kuva, joka näyttää upotusluokittelijan viidelle sanajonolle.](../../../../../translated_images/fi/embedding-classifier-example.b77f021a7ee67eee.webp)

Luokittelijaneuroverkkomme alkaa upotuskerroksella, sitten aggregaatiokerroksella ja sen päällä lineaarisella luokittelijalla:


In [2]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, x):
        x = self.embedding(x)
        x = torch.mean(x,dim=1)
        return self.fc(x)

### Käsittely muuttuvan kokoisia sekvenssejä

Tämän arkkitehtuurin seurauksena minibatchit verkkoomme täytyy luoda tietyllä tavalla. Edellisessä osiossa, kun käytimme bag-of-words-menetelmää, kaikki BoW-tensorit minibatchissa olivat saman kokoisia, `vocab_size`, riippumatta tekstisekvenssin todellisesta pituudesta. Kun siirrymme sanavektoreihin, päädymme tilanteeseen, jossa jokaisessa tekstinäytteessä on vaihteleva määrä sanoja, ja näytteitä yhdistettäessä minibatcheiksi meidän täytyy käyttää täydennystä.

Tämä voidaan tehdä käyttämällä samaa tekniikkaa, jossa `collate_fn`-funktio annetaan tietolähteelle:


In [3]:
def padify(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # first, compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)

### Upotetun luokittelijan kouluttaminen

Nyt kun olemme määrittäneet sopivan datalaturin, voimme kouluttaa mallin käyttämällä edellisessä osiossa määriteltyä koulutustoimintoa:


In [4]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=1, epoch_size=25000)

3200: acc=0.6415625
6400: acc=0.6865625
9600: acc=0.7103125
12800: acc=0.726953125
16000: acc=0.739375
19200: acc=0.75046875
22400: acc=0.7572321428571429


(0.889799795315499, 0.7623160588611644)

> **Huom**: Tässä harjoittelemme vain 25 000 tietueella (alle yksi kokonainen epookki) ajan säästämiseksi, mutta voit jatkaa harjoittelua, kirjoittaa funktion useiden epookkien harjoitteluun ja kokeilla oppimisnopeuden parametria saavuttaaksesi korkeamman tarkkuuden. Sinun pitäisi pystyä saavuttamaan noin 90 % tarkkuus.


### EmbeddingBag-kerros ja muuttuvan pituisen sekvenssin esitys

Edellisessä arkkitehtuurissa meidän täytyi täyttää kaikki sekvenssit samanpituisiksi, jotta ne sopisivat minibatchiin. Tämä ei ole tehokkain tapa esittää muuttuvan pituisia sekvenssejä – toinen lähestymistapa olisi käyttää **offset**-vektoria, joka sisältää kaikkien yhteen suureen vektoriin tallennettujen sekvenssien offsetit.

![Kuva, joka näyttää offset-sekvenssiesityksen](../../../../../translated_images/fi/offset-sequence-representation.eb73fcefb29b46ee.webp)

> **Huom**: Yllä olevassa kuvassa esitetään merkkijonosekvenssi, mutta esimerkissämme työskentelemme sanasekvenssien kanssa. Yleinen periaate sekvenssien esittämisestä offset-vektorilla pysyy kuitenkin samana.

Offset-esityksen kanssa työskentelyyn käytämme [`EmbeddingBag`](https://pytorch.org/docs/stable/generated/torch.nn.EmbeddingBag.html)-kerrosta. Se on samankaltainen kuin `Embedding`, mutta se ottaa syötteenä sisältövektorin ja offset-vektorin, ja siihen sisältyy myös keskiarvokerros, joka voi olla `mean`, `sum` tai `max`.

Tässä on muokattu verkko, joka käyttää `EmbeddingBag`-kerrosta:


In [5]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.EmbeddingBag(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, text, off):
        x = self.embedding(text, off)
        return self.fc(x)

Valmistaaksemme tietojoukon koulutusta varten, meidän täytyy tarjota muunnostoiminto, joka valmistaa offset-vektorin:


In [6]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1])) for t in b]
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)

Huomaa, että toisin kuin kaikissa aiemmissa esimerkeissä, verkkomme hyväksyy nyt kaksi parametria: datavektorin ja offset-vektorin, jotka ovat eri kokoisia. Samoin datalaturimme tarjoaa meille 3 arvoa 2 sijaan: sekä teksti- että offset-vektorit tarjotaan ominaisuuksina. Siksi meidän on hieman mukautettava koulutustoimintoamme huolehtimaan tästä:


In [7]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)

def train_epoch_emb(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.CrossEntropyLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    loss_fn = loss_fn.to(device)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,text,off in dataloader:
        optimizer.zero_grad()
        labels,text,off = labels.to(device), text.to(device), off.to(device)
        out = net(text, off)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count


train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6153125
6400: acc=0.6615625
9600: acc=0.6932291666666667
12800: acc=0.715078125
16000: acc=0.7270625
19200: acc=0.7382291666666667
22400: acc=0.7486160714285715


(22.771553103007037, 0.7551983365323096)

## Semanttiset upotukset: Word2Vec

Edellisessä esimerkissämme mallin upotuskerros oppi kartoittamaan sanat vektorimuotoon, mutta tällä esityksellä ei ollut juurikaan semanttista merkitystä. Olisi hyödyllistä oppia sellainen vektorimuoto, jossa samankaltaiset sanat tai synonyymit vastaisivat toisiaan lähellä olevia vektoreita jonkin vektorietäisyyden (esim. euklidinen etäisyys) perusteella.

Tämän saavuttamiseksi meidän täytyy esikouluttaa upotusmallimme suurella tekstikokoelmalla tietyllä tavalla. Yksi ensimmäisistä tavoista kouluttaa semanttisia upotuksia tunnetaan nimellä [Word2Vec](https://en.wikipedia.org/wiki/Word2vec). Se perustuu kahteen pääarkkitehtuuriin, joita käytetään sanojen hajautetun esityksen tuottamiseen:

 - **Jatkuva sanapussimalli** (CBoW) — tässä arkkitehtuurissa koulutamme mallin ennustamaan sanan ympäröivän kontekstin perusteella. Annetulla ngrammilla $(W_{-2},W_{-1},W_0,W_1,W_2)$ mallin tavoitteena on ennustaa $W_0$ käyttäen $(W_{-2},W_{-1},W_1,W_2)$.
 - **Jatkuva skip-gram** on CBoW:n vastakohta. Malli käyttää ympäröivää kontekstisanan ikkunaa ennustaakseen nykyisen sanan.

CBoW on nopeampi, kun taas skip-gram on hitaampi, mutta se edustaa harvinaisia sanoja paremmin.

![Kuva, joka näyttää sekä CBoW- että Skip-Gram-algoritmit sanojen muuntamiseksi vektoreiksi.](../../../../../translated_images/fi/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

Kokeillaksemme Word2Vec-upotusta, joka on esikoulutettu Google News -aineistolla, voimme käyttää **gensim**-kirjastoa. Alla etsimme sanoja, jotka ovat lähimpänä sanaa 'neural'.

> **Note:** Kun luot sanavektoreita ensimmäistä kertaa, niiden lataaminen voi kestää jonkin aikaa!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [9]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


Voimme myös laskea sanasta vektorijoukoja, joita käytetään luokittelumallin koulutuksessa (näytämme selkeyden vuoksi vain vektorin ensimmäiset 20 komponenttia):


In [10]:
w2v.word_vec('play')[:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

Hieno asia semanttisissa upotuksissa on, että voit manipuloida vektoriin koodausta muuttaaksesi semantiikkaa. Esimerkiksi voimme pyytää löytämään sanan, jonka vektoriedustus olisi mahdollisimman lähellä sanoja *kuningas* ja *nainen*, ja mahdollisimman kaukana sanasta *mies*:


In [10]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

Sekä CBoW että Skip-Grams ovat "ennustavia" upotuksia, koska ne ottavat huomioon vain paikalliset kontekstit. Word2Vec ei hyödynnä globaalia kontekstia.

**FastText** rakentuu Word2Vecin pohjalle oppimalla vektoriedustuksia jokaiselle sanalle sekä sanan sisällä oleville merkkien n-grammeille. Näiden edustusten arvot keskiarvoistetaan yhdeksi vektoriksi jokaisessa harjoitusvaiheessa. Vaikka tämä lisää merkittävästi laskentaa esikoulutuksessa, se mahdollistaa sanaupotusten koodata osasanan tietoa.

Toinen menetelmä, **GloVe**, hyödyntää yhteisesiintymismatriisin ideaa ja käyttää neuroverkkomenetelmiä hajottaakseen yhteisesiintymismatriisin ilmeikkäämmiksi ja epälineaarisiksi sanavektoreiksi.

Voit kokeilla esimerkkiä vaihtamalla upotukset FastTextiin ja GloVeen, sillä gensim tukee useita erilaisia sanaupotusmalleja.


## Esikoulutettujen upotusten käyttäminen PyTorchissa

Voimme muokata yllä olevaa esimerkkiä esitäyttääksemme upotuskerroksen matriisin semanttisilla upotuksilla, kuten Word2Vecillä. Meidän on otettava huomioon, että esikoulutetun upotuksen ja tekstikorpuksemme sanastot eivät todennäköisesti vastaa toisiaan, joten alustamme puuttuvien sanojen painot satunnaisilla arvoilla:


In [11]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

net = EmbedClassifier(vocab_size,embed_size,len(classes))

print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab.get_itos()):
    try:
        net.embedding.weight[i].data = torch.tensor(w2v.get_vector(w))
        found+=1
    except:
        net.embedding.weight[i].data = torch.normal(0.0,1.0,(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")
net = net.to(device)

Embedding size: 300
Populating matrix, this will take some time...Done, found 41080 words, 54732 words missing


Nyt harjoitellaan malliamme. Huomaa, että mallin harjoittamiseen kuluva aika on huomattavasti pidempi kuin edellisessä esimerkissä, johtuen suuremmasta upotuskerroksen koosta ja siten paljon suuremmasta parametrien määrästä. Lisäksi tämän vuoksi saatamme joutua harjoittelemaan malliamme useammilla esimerkeillä, jos haluamme välttää ylioppimisen.


In [12]:
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6359375
6400: acc=0.68109375
9600: acc=0.7067708333333333
12800: acc=0.723671875
16000: acc=0.73625
19200: acc=0.7463541666666667
22400: acc=0.7560714285714286


(214.1013875559821, 0.7626759436980166)

Meidän tapauksessamme emme näe suurta tarkkuuden kasvua, mikä johtuu todennäköisesti hyvin erilaisista sanastoista.  
Ongelman ratkaisemiseksi, joka liittyy erilaisiin sanastoihin, voimme käyttää jotakin seuraavista ratkaisuista:  
* Kouluttaa word2vec-malli uudelleen meidän sanastollamme  
* Ladata datasetimme sanastolla, joka on peräisin valmiiksi koulutetusta word2vec-mallista. Sanasto, jota käytetään datasetin lataamiseen, voidaan määrittää latauksen aikana.  

Jälkimmäinen lähestymistapa vaikuttaa helpommalta, erityisesti koska PyTorchin `torchtext`-kehys sisältää sisäänrakennetun tuen upotuksille. Voimme esimerkiksi luoda sanaston, joka perustuu GloVeen, seuraavalla tavalla:  


In [14]:
vocab = torchtext.vocab.GloVe(name='6B', dim=50)

100%|█████████▉| 399999/400000 [00:15<00:00, 25411.14it/s]


Ladattu sanasto sisältää seuraavat perustoiminnot:
* `vocab.stoi`-sanakirjan avulla voimme muuntaa sanan sen sanakirjaindeksiksi
* `vocab.itos` tekee päinvastoin - muuntaa numeron sanaksi
* `vocab.vectors` on upotusvektoreiden taulukko, joten saadaksemme sanan `s` upotuksen meidän täytyy käyttää `vocab.vectors[vocab.stoi[s]]`

Tässä on esimerkki upotusten käsittelystä, joka havainnollistaa yhtälöä **kind-man+woman = queen** (jouduin säätämään kerrointa hieman, jotta se toimisi):


In [15]:
# get the vector corresponding to kind-man+woman
qvec = vocab.vectors[vocab.stoi['king']]-vocab.vectors[vocab.stoi['man']]+1.3*vocab.vectors[vocab.stoi['woman']]
# find the index of the closest embedding vector 
d = torch.sum((vocab.vectors-qvec)**2,dim=1)
min_idx = torch.argmin(d)
# find the corresponding word
vocab.itos[min_idx]

'queen'

Jotta voimme kouluttaa luokittelijan näiden upotusten avulla, meidän on ensin koodattava tietoaineistomme GloVe-sanakirjan avulla:


In [16]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1],voc=vocab)) for t in b] # pass the instance of vocab to encode function!
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

Kuten olemme nähneet yllä, kaikki vektoriesitykset tallennetaan `vocab.vectors`-matriisiin. Tämä tekee painojen lataamisesta upotuskerroksen painoihin erittäin helppoa yksinkertaisella kopioinnilla:


In [17]:
net = EmbedClassifier(len(vocab),len(vocab.vectors[0]),len(classes))
net.embedding.weight.data = vocab.vectors
net = net.to(device)

Nyt koulutetaan mallimme ja katsotaan, saammeko parempia tuloksia:


In [18]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6271875
6400: acc=0.68078125
9600: acc=0.7030208333333333
12800: acc=0.71984375
16000: acc=0.7346875
19200: acc=0.7455729166666667
22400: acc=0.7529464285714286


(35.53972978646833, 0.7575175943698017)

Yksi syy siihen, miksi emme näe merkittävää tarkkuuden kasvua, johtuu siitä, että jotkin sanastomme sanat puuttuvat esikoulutetusta GloVe-sanakirjasta, ja siksi ne käytännössä jätetään huomiotta. Tämän ongelman voittamiseksi voimme kouluttaa omat upotuksemme omalla aineistollamme.


## Kontekstuaaliset upotukset

Yksi perinteisten esikoulutettujen upotusten, kuten Word2Vecin, merkittävä rajoitus on sanan merkityksen erottelun ongelma. Vaikka esikoulutetut upotukset voivat osittain vangita sanojen merkityksen kontekstissa, jokainen sanan mahdollinen merkitys koodataan samaan upotukseen. Tämä voi aiheuttaa ongelmia jatkomalleissa, koska monet sanat, kuten sana 'play', voivat tarkoittaa eri asioita riippuen siitä, missä kontekstissa niitä käytetään.

Esimerkiksi sana 'play' tarkoittaa näissä kahdessa lauseessa hyvin eri asioita:
- Kävin katsomassa **näytelmää** teatterissa.
- John haluaa **leikkiä** ystäviensä kanssa.

Yllä olevat esikoulutetut upotukset edustavat molempia sanan 'play' merkityksiä samassa upotuksessa. Tämän rajoituksen voittamiseksi meidän täytyy rakentaa upotuksia, jotka perustuvat **kielimalliin**, joka on koulutettu suurella tekstikorpuksella ja *tietää*, miten sanoja voidaan yhdistää eri konteksteissa. Kontekstuaalisten upotusten käsittely on tämän opetusmateriaalin ulkopuolella, mutta palaamme niihin, kun käsittelemme kielimalleja seuraavassa osiossa.



---

**Vastuuvapauslauseke**:  
Tämä asiakirja on käännetty käyttämällä tekoälypohjaista käännöspalvelua [Co-op Translator](https://github.com/Azure/co-op-translator). Vaikka pyrimme tarkkuuteen, huomioithan, että automaattiset käännökset voivat sisältää virheitä tai epätarkkuuksia. Alkuperäistä asiakirjaa sen alkuperäisellä kielellä tulisi pitää ensisijaisena lähteenä. Kriittisen tiedon osalta suositellaan ammattimaista ihmiskäännöstä. Emme ole vastuussa väärinkäsityksistä tai virhetulkinnoista, jotka johtuvat tämän käännöksen käytöstä.
